# 🛰️ Sentinel-1 SAR Data Analysis using Google Earth Engine  

This notebook performs **Sentinel-1 SAR data processing** for water body detection using **NDWI (Normalized Difference Water Index)** in your selected **Area of Interest (AOI)**.  

---

## 🚀 How to Run the Code  

1. **Run each code cell one by one (top to bottom).**  
   - The first cell installs and imports all necessary libraries like `geemap`, `ee`, and `folium`.  

2. **Authenticate your Google Earth Engine account.**  
   - When prompted, click the link shown in the output.  
   - Sign in with your Google account.  
   - Copy the authentication code and paste it back into the cell.  

3. **Draw your AOI (Area of Interest).**  
   - Wait for the map to load.  
   - Use the drawing tools (rectangle or polygon icon) on the left side of the map to select your study area.  

4. **Set the Date Range.**  
   - Use the dropdown or input boxes to select your desired **start date** and **end date** for Sentinel-1 data collection.  

5. **Run the Analysis.**  
   - The code will automatically filter Sentinel-1 images for your AOI and date range.  
   - It will compute the NDWI index and display a processed map of water bodies.  

6. **View the Results.**  
   - The results will be shown on an interactive map below the code cell.  
   - A time-series chart (line graph) of NDWI variation may also appear if included in the notebook.  

7. **Export the Output (Optional).**  
   - If export code is available, you can export the processed raster to your Google Drive.  

---

## 🧠 Notes  

- If the map does not appear, click **“Runtime → Restart and Run All”** in the Colab menu.  
- Ensure you are logged into the same Google account linked to Earth Engine.  
- Larger AOIs or long time ranges may take extra processing time.  

---

## 📦 Libraries Used  

- **geemap** – for interactive GEE + folium mapping  
- **ee (Earth Engine API)** – for accessing satellite data  
- **folium** – for rendering maps  
- **matplotlib / pandas** – for data visualization and analysis

In [ ]:
!pip install ipywidgets geemap
from ipywidgets import widgets, interact, Layout
import geemap
import ee
import datetime

In [ ]:
ee.Authenticate()
ee.Initialize(project="ee-sammaglin1971")

In [ ]:
# Define start and end dates
start_date = datetime.date(2021, 7, 1)
end_date = datetime.date(2021, 7, 15)

print(f"Start Date: {start_date}")
print(f"End Date: {end_date}")

In [ ]:
Map = geemap.Map(center=[25.6, 85.3], zoom=7)
Map.add_basemap("SATELLITE")
Map.add_draw_control()  # user draws AOI
Map

In [ ]:
def run_analysis(start_date, end_date):
    aoi = Map.user_roi
    if aoi is None:
        print("Draw your AOI first!")
        return

    # Pre and Post flood Sentinel-1
    pre_flood = ee.ImageCollection("COPERNICUS/S1_GRD") \
        .filterBounds(aoi) \
        .filterDate(str(start_date), str(start_date + datetime.timedelta(days=7))) \
        .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV")) \
        .select("VV").mean()

    post_flood = ee.ImageCollection("COPERNICUS/S1_GRD") \
        .filterBounds(aoi) \
        .filterDate(str(end_date), str(end_date + datetime.timedelta(days=7))) \
        .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV")) \
        .select("VV").mean()

    # Simple analysis: difference
    flood_diff = post_flood.subtract(pre_flood)
    flood_mask = flood_diff.lt(-2).clip(aoi)

    Map.addLayer(pre_flood, {"min":-25,"max":0}, "Pre-Flood VV")
    Map.addLayer(post_flood, {"min":-25,"max":0}, "Post-Flood VV")
    Map.addLayer(flood_mask.updateMask(flood_mask), {"palette":["blue"]}, "Flooded Areas")
    display(Map)

In [ ]:
run_analysis(start_date, end_date)